In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import urllib.request

import sys

In [2]:
sys.path.append('/Users/nicksubic/Documents/flatiron/phase_1/nyc-mhtn-ds-091420-lectures/capstone/Clothing_Recommender/src/')
sys.path.append('/Users/nicksubic/.wdm/drivers/chromedriver/mac64/87.0.4280.88/')

In [3]:
import data

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/nicksubic/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache
 


In [5]:
# Set a "base" URL to append onto
base_url = "https://www.grailed.com/categories/tops"

chrome_options = webdriver.ChromeOptions()

driver.get(base_url)
timeout = 30

try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='feed-item']")))

except TimeoutException:
    print("Timed out")
    driver.quit()

In [6]:
# skip elements at the top of the page
results = driver.find_elements_by_xpath('//div[@class="FiltersInstantSearch"]//div[@class="feed-item"]')

ScrollNumber=1

# Start the scroll
for i in range(0,ScrollNumber):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)

# # Run again to see total number of results the scraper is going to get. Should be the same as ListingNumber above
results = driver.find_elements_by_xpath('//div[@class="FiltersInstantSearch"]//div[@class="feed-item"]')
len(results)

80

In [7]:
def make_dataframe(results):
    ''''''

    Name = []
    Designer = []
    Price = []
    NewPrice = []
    OldPrice = []
    Size = []
    Time = []
    LastBump = []
    Link = []
    Image = []

    for i, result in enumerate(results):

        Designer.append(result.find_element_by_class_name("listing-designer").text)
        Name.append(result.find_element_by_class_name("listing-title").text)
        
        try:
            Price.append(result.find_element_by_xpath('.//p[@class="sub-title original-price"]').text)
            NewPrice.append(np.nan)
            OldPrice.append(np.nan)
        except NoSuchElementException:
            NewPrice.append(result.find_element_by_xpath('.//p[@class="sub-title new-price"]').text)
            OldPrice.append(result.find_element_by_xpath('.//p[@class="sub-title original-price strike-through"]').text)
            Price.append(np.nan)

        Size.append(result.find_element_by_xpath('.//p[@class="listing-size sub-title"]').text)

        Time.append(result.find_element_by_xpath(".//span[@class='date-ago']").text)

        try:
            LastBump.append(result.find_element_by_xpath(".//span[@class='strike-through']").text)
        except NoSuchElementException:
            LastBump.append(np.nan)

        Link.append(result.find_element_by_xpath('./a').get_attribute("href"))

        grailed_dict = {'Name': Name, 
                    'Designer': Designer, 
                    'Price': Price, 
                    'NewPrice': NewPrice, 
                    'OldPrice': OldPrice, 
                    'Size': Size, 
                    'Time': Time, 
                    'LastBump': LastBump, 
                    'Link': Link}
    return pd.DataFrame(grailed_dict)

In [8]:
df = make_dataframe(results)

In [49]:
images = []
imgurl= []
for result in results:
    images.append(result.find_element_by_xpath('//div[@class="listing-cover-photo "]//div[@class="lazyload-wrapper"]/img'))
# for image in images:
#     imgurl.append(image.get_attribute("src"))
# for image in images:
#     print(image.text)

In [9]:
# images = []
# imgurl= []
# for result in results:
#     images.append(result.find_elements_by_css_selector(By.TAG_NAME, 'img'))
# for image in images:
#     imgurl.append(image.get_attribute("src"))

In [119]:
# image = result.find_element_by_xpath('//div[@class="listing-cover-photo "]//div[@class="lazyload-wrapper"]//img').get_attribute("src")


In [10]:
# for result in results:
#     images = result.find_elements_by_xpath('//div[@class="listing-cover-photo "]')
# # # image = results.find_element_by_tag_name("img")
# src = []
# for image in images:
#     src.append(image.find_element_by_tag_name('img'))
#     # for i, url in enumerate(src):
    #     urllib.request.urlretrieve(url, f'src/images/{i}.jpg')


In [11]:
chrome_options.add_argument('--headless')
# driver = webdriver.Chrome(options=chrome_options)

# Lists to append everything to
UserName=[]
Sold=[]
Feedback=[]
CurrentListings=[]
Description=[]
ProfileLink=[]
FeedBack=[]
FeedbackLink=[]
FollowerCount=[]
FullSize=[]
PostedTime=[]
BumpedTime=[]
Location=[]
Transactions=[]

### Start Process. Link is a list of links from ItemDF
for i, link in enumerate(df.Link):

    driver.get(link)

    image = driver.find_element_by_xpath('//div[@class="-image-wrapper"]')
    src = image.find_element_by_tag_name('img').get_attribute('src')
    urllib.request.urlretrieve(src, f'src/images/{i}.jpg')

    try:
        UserName.append(driver.find_element_by_xpath('//span[@class="-username"]').text)
    except NoSuchElementException:
        UserName.append(np.nan)

    try:
        Sold.append(driver.find_element_by_xpath('//a[@class="-link"]/span[2]').text)
    except NoSuchElementException:
        Sold.append(np.nan)

    try:
        FeedBack.append(driver.find_element_by_xpath('//span[@class="-feedback-count"]').text)
    except NoSuchElementException:
        FeedBack.append(np.nan)

    try:
        CurrentListings.append(driver.find_element_by_xpath('//a[@class="-for-sale-link"]').text)
    except NoSuchElementException:
        CurrentListings.append(np.nan)

    try:
        Description.append(driver.find_element_by_xpath('//div[@class="listing-description"]').text)
    except NoSuchElementException:
        Description.append(np.nan)

    try:
        ProfileLink.append(driver.find_element_by_xpath('//span[@class="Username"]/a').get_attribute("href"))
    except NoSuchElementException:
        ProfileLink.append(np.nan)

    try:
        FeedbackLink.append(driver.find_element_by_xpath('//div[@class="-details"]/a').get_attribute("href"))
    except NoSuchElementException:
        FeedbackLink.append(np.nan)

    try:
        FollowerCount.append(driver.find_element_by_xpath('//p[@class="-follower-count"]').text)
    except NoSuchElementException:
        FollowerCount.append(np.nan)

    try:
        FullSize.append(driver.find_element_by_xpath('//h2[@class="listing-size sub-title"]').text)
    except NoSuchElementException:
        FullSize.append(np.nan)

    try:
        PostedTime.append(driver.find_element_by_xpath('//div[@class="-metadata"]/span[2]').text)
    except NoSuchElementException:
        PostedTime.append(np.nan)

    try:
        BumpedTime.append(driver.find_element_by_xpath('//div[@class="-metadata"]/span[4]').text)
    except NoSuchElementException:
        BumpedTime.append(np.nan)

    try:
        Location.append(driver.find_element_by_xpath('//label[@class="--label"]').text)
    except NoSuchElementException:
        Location.append(np.nan)

    # image = driver.find_element_by_class_name("-image _selected")
    # src = image.get_attribute('src')
    # urllib.request.urlretrieve(src, f'src/images/{i}.jpg')

In [12]:
page_dict = {'Username': UserName,'Sold': Sold,'Feedback': FeedBack,'CurrentListings': CurrentListings, 'Description': Description, 'ProfileLink': ProfileLink, 'FeedbackLink': FeedbackLink, 'FollowerCount': FollowerCount,'FullSize': FullSize,'PostedTime': PostedTime,'BumpedTime': BumpedTime, 'Location': Location}
SellerDF = pd.DataFrame(page_dict)

In [13]:
df = pd.concat([df, SellerDF], axis = 1)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 21 columns):
Name               80 non-null object
Designer           80 non-null object
Price              73 non-null object
NewPrice           7 non-null object
OldPrice           7 non-null object
Size               80 non-null object
Time               80 non-null object
LastBump           4 non-null object
Link               80 non-null object
Username           80 non-null object
Sold               80 non-null object
Feedback           80 non-null object
CurrentListings    80 non-null object
Description        80 non-null object
ProfileLink        80 non-null object
FeedbackLink       80 non-null object
FollowerCount      80 non-null object
FullSize           80 non-null object
PostedTime         80 non-null object
BumpedTime         4 non-null object
Location           80 non-null object
dtypes: object(21)
memory usage: 13.2+ KB


In [ ]:
driver.close()